In [61]:
import pandas as pd
import tqdm.notebook as tqdm
from pandarallel import pandarallel

In [66]:
import glob

rankings = pd.concat(
    (pd.read_csv(
        path,
        header=None,
        names=["pageURL", "pageRank", "avgDuration"],
    )
    for path in glob.glob("data/AMPLab/1node/rankings/*.csv")),
    axis=0,
    ignore_index=True, 
)

In [3]:
rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pageURL      1200 non-null   object
 1   pageRank     1200 non-null   int64 
 2   avgDuration  1200 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 28.2+ KB


In [17]:
query_1 = rankings.loc[rankings.pageRank > 100]
len(query_1)

45

In [68]:
uservisits = pd.concat(
    (pd.read_csv(
        path,
        header=None,
        names=["sourceIP", "destinationURL", "visitDate", "adRevenue", "UserAgent", "cCode", "lCode", "searchWord", "duration"],
        parse_dates=["visitDate"],
    )
    for path in glob.glob("data/AMPLab/1node/uservisits/*.csv")[:200]),
    axis=0,
    ignore_index=True, 
)

In [58]:
%%time
query_2 = (
uservisits
    .groupby(uservisits.sourceIP.str.slice(0, 7))
    .adRevenue
    .sum()
)
query_2

CPU times: user 14.3 s, sys: 1.22 s, total: 15.5 s
Wall time: 15.5 s


sourceIP
1.0.0.1     0.083036
1.0.1.1     0.916992
1.0.1.5     0.423281
1.0.10.     2.862430
1.0.100     1.203593
             ...    
99.99.5    13.071833
99.99.6    10.827506
99.99.7    14.079676
99.99.8    15.141336
99.99.9    14.524313
Name: adRevenue, Length: 339624, dtype: float64

In [62]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [64]:
%%time

query_2 = (
uservisits
    .parallel_groupby(uservisits.sourceIP.str.slice(0, 7))
    .adRevenue
    .sum()
)
query_2

AttributeError: 'DataFrame' object has no attribute 'parallel_groupby'

In [26]:
(rankings
    .merge(uservisits, left_on="pageURL", right_on="destinationURL", how="left")
    .query("visitDate > '1980-01-01' and visitDate < '1980-04-01'")
)

,pageURL,pageRank,avgDuration,sourceIP,destinationURL,visitDate,adRevenue,UserAgent,cCode,lCode,searchWord,duration
100,jrcxilplavgpxejqveqahdsfbxpwjirgzfkuwtiphpogiy...,56,83,16.28.44.171,jrcxilplavgpxejqveqahdsfbxpwjirgzfkuwtiphpogiy...,1980-03-16,0.371041,Ovsqbqdyxdqv/5.3,PRI,PRI-ES,tikoloshe,4.0
561,lfpldxezkcgnfshiowiigogrorlvvqawodwdtxsze,29,52,109.146.18.47,lfpldxezkcgnfshiowiigogrorlvvqawodwdtxsze,1980-02-13,0.917136,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,LTU,LTU-LT,phrenic,3.0
702,dtvrjhbgchtdrjuoqnjniaxp,24,36,78.97.207.17,dtvrjhbgchtdrjuoqnjniaxp,1980-01-27,0.101126,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,USA,USA-ES,unpurely,7.0
833,ehegqevxsvojrlgljajtjhpnbuaxxnmoyanxqogjxuk,27,74,149.70.228.139,ehegqevxsvojrlgljajtjhpnbuaxxnmoyanxqogjxuk,1980-01-22,0.791280,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,USA,USA-EN,equivale,3.0
1004,bxpbjhrpilgpkkjzkhqxgcsanpbqxvpcldlpymesckixcu...,144,54,67.131.48.189,bxpbjhrpilgpkkjzkhqxgcsanpbqxvpcldlpymesckixcu...,1980-01-10,0.922786,Mozilla/5.0 (Windows; U; Windows NT 5.1)Gecko/...,RUS,RUS-RU,siderites,2.0
1128,pubgiqursfdfpvcrdmsddekelzmaycotadecdwandyixpb...,44,72,123.113.161.232,pubgiqursfdfpvcrdmsddekelzmaycotadecdwandyixpb...,1980-03-28,0.163713,iPhone 3.0: Mozilla/5.0 (iPhone; U; CPU iPhone...,NIC,NIC-ES,hcb,1.0
1354,dikcsejbwtcwrfjwyxypwbsldaxnahaaysjnvhoj,27,56,202.132.171.66,dikcsejbwtcwrfjwyxypwbsldaxnahaaysjnvhoj,1980-02-19,0.306470,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,USA,USA-ES,tattooers,7.0
1721,cmhrxndxefmmmpldfwgdldyoabiav,21,24,13.95.33.84,cmhrxndxefmmmpldfwgdldyoabiav,1980-01-30,0.999274,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,THA,THA-TH,snipping,2.0
1989,dagtwwybivyiuxmkhbxkkpvybycyionubrxfqxjdmpsjra...,177,74,139.98.168.53,dagtwwybivyiuxmkhbxkkpvybycyionubrxfqxjdmpsjra...,1980-02-05,0.504104,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,CHN,CHN-ZH,Neper,9.0
1998,emucailxlqlqazqrupsaphhmsgqifscofkyhybngvcbxda,134,89,142.58.244.50,emucailxlqlqazqrupsaphhmsgqifscofkyhybngvcbxda,1980-03-27,0.579072,Mozilla/5.0 (Windows; U; Windows NT 5.2) Apple...,CAN,CAN-FR,grece,4.0


In [67]:
query_3 = (
rankings
    .merge(uservisits, left_on="pageURL", right_on="destinationURL", how="left")
    .query("visitDate > '1980-01-01' and visitDate < '1980-04-01'")
    .groupby("sourceIP")
    .agg({"pageRank": "mean", "adRevenue": "sum"})
    .sort_values("adRevenue", ascending=False)
    .head(10)
)
query_3

,pageRank,adRevenue
sourceIP,,
244.228.241.205,26.0,1.751174
160.0.52.128,24.0,1.285921
143.158.25.22,23.5,1.028893
107.241.178.250,38.0,0.999996
92.190.11.83,37.0,0.999993
212.131.228.15,20.0,0.999993
194.231.243.66,37.0,0.999993
57.45.1.141,19.0,0.999988
182.251.108.194,21.0,0.999985


In [37]:
crawl = pd.concat(
    (pd.read_table(
        path,
        sep="dasdsadsa",
        header=None,
        engine="python",
        names=["pageSource"],
    )
    for path in glob.glob("data/AMPLab/tiny/crawl/*.csv")),
    axis=0,
    ignore_index=True, 
)

In [40]:
import re
url_regex = re.compile("(?P<url>https?://[^\s]+)")

def extract_url(line):
    match = url_regex.search(line)
    if match:
        return match.group("url")
    else:
        return None
crawl["destPage"] = crawl.pageSource.apply(extract_url)

In [65]:
(crawl
    .pageSource
    .parallel_apply(extract_url)
    .value_counts()
) 

http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">                         84
http://pagead2.googlesyndication.com/pagead/show_ads.js">                         59
http://www.w3.org/1999/xhtml"                                                     53
https://ssl."                                                                     51
http://www.w3.org/1999/xhtml">                                                    43
                                                                                  ..
http://www.HireMilitary.com/2009/12/'                                              1
http://www.hiqa.com/images/MenuStartCurve.jpg"                                     1
http://gravatar.com"><b>gravatar</b></a>                                           1
http://www.him-motion.com/#/pageproduit/146">                                      1
http://kandai-syouken.enjyuku-investudent.com/archives/img/gakusei02.bmp"><img     1
Name: pageSource, Length: 8822, dtype: int64

In [50]:
(crawl
    .groupby("destPage", as_index=False)
    .size()
    .sort_values("size", ascending=False)
)

,destPage,size
8501,http://www.w3.org/TR/xhtml1/DTD/xhtml1-transit...,84
2377,http://pagead2.googlesyndication.com/pagead/sh...,59
8483,"http://www.w3.org/1999/xhtml""",53
8794,"https://ssl.""",51
8484,"http://www.w3.org/1999/xhtml"">",43
...,...,...
2993,http://tom.acrewoods.net/tag/democracy/,1
2992,"http://toddwiley.com/feed/"">Articles</a><br",1
2990,"http://tocando.org/blog/category/idolos/""",1
2989,http://tocando.org/Content/images/tocando-org-...,1
